In [48]:
import numpy as np
import pandas as pd

data = np.genfromtxt('210504 Solar Load.csv',delimiter=',',skip_header=True,skip_footer=True)
datat = np.transpose(data)
specific_solar_load = datat[2]

In [50]:
print(specific_solar_load)

solar_pv_capacity = 200.0
BESS_kw = 100.0
BESS_kwh = 238.0
BESS_RTE = 0.8
BESS_roundtrip_efficiency = 0.8

[0. 0. 0. ... 0. 0. 0.]


In [52]:
checkLine = 14
total_solar_pv_generation = specific_solar_load*solar_pv_capacity
print(total_solar_pv_generation[checkLine])
customer_demand = 50.0
SPVS_to_load = np.zeros(shape=8760)
available_solar_for_BESS = np.zeros(shape=8760)
BESS_charge_from_solar_pv = np.zeros(shape=8760)
actual_solar_pv_supply_to_BESS = np.zeros(shape=8760)
solar_pv_to_be_curtailed = np.zeros(shape=8760)
BESS_discharge_to_load = np.zeros(shape=8760)
DG_supply = np.zeros(shape=8760)
BESS_balance = BESS_kwh
charging = False
discharging = True

for i in range(0,8760):
    SPVS_to_load[i] = min(customer_demand,total_solar_pv_generation[i])
    available_solar_for_BESS[i] = min(BESS_kw,max(0,total_solar_pv_generation[i]- customer_demand))
    if i%24==0:
        discharging = True
        charging = False
    if discharging == True:
        if BESS_balance == 0:
            charging = True
            discharging = False
        else:
            BESS_discharge_to_load[i] = min(customer_demand,BESS_balance)
            BESS_balance = BESS_balance - BESS_discharge_to_load[i]
    if charging==True: 
        BESS_charge_from_solar_pv[i] = min((available_solar_for_BESS[i]*BESS_roundtrip_efficiency),(BESS_kwh-BESS_balance))
        BESS_balance = BESS_balance + BESS_charge_from_solar_pv[i]
        if BESS_balance >= BESS_kwh:
            charging = False
    #print(i,BESS_balance,charging, BESS_charge_from_solar_pv[i],BESS_discharge_to_load[i])
        

    actual_solar_pv_supply_to_BESS[i] = BESS_charge_from_solar_pv[i]/BESS_RTE
    solar_pv_to_be_curtailed[i] = total_solar_pv_generation[i]-SPVS_to_load[i]-actual_solar_pv_supply_to_BESS[i]
    DG_supply[i] = customer_demand - SPVS_to_load[i] - BESS_discharge_to_load[i]

print('Calculation Finished')

8.33803760000001 True 0.0 0.0
8107 58.33803760000001 True 0.0 0.0
8108 58.33803760000001 True 0.0 0.0
8109 58.33803760000001 True 0.0 0.0
8110 58.33803760000001 True 0.0 0.0
8111 58.33803760000001 True 0.0 0.0
8112 8.338037600000007 False 0.0 50.0
8113 0.0 False 0.0 8.338037600000007
8114 0.0 True 0.0 0.0
8115 0.0 True 0.0 0.0
8116 0.0 True 0.0 0.0
8117 0.0 True 0.0 0.0
8118 0.0 True 0.0 0.0
8119 0.0 True 0.0 0.0
8120 11.645779840000001 True 11.645779840000001 0.0
8121 45.32034176 True 33.674561919999995 0.0
8122 101.30056207999999 True 55.980220319999994 0.0
8123 154.15899263999998 True 52.85843055999999 0.0
8124 213.98720672 True 59.82821408 0.0
8125 238.0 False 24.01279328000001 0.0
8126 238.0 False 0.0 0.0
8127 238.0 False 0.0 0.0
8128 238.0 False 0.0 0.0
8129 238.0 False 0.0 0.0
8130 238.0 False 0.0 0.0
8131 238.0 False 0.0 0.0
8132 238.0 False 0.0 0.0
8133 238.0 False 0.0 0.0
8134 238.0 False 0.0 0.0
8135 238.0 False 0.0 0.0
8136 188.0 False 0.0 50.0
8137 138.0 False 0.0 50.0
813

In [54]:
print(SPVS_to_load[checkLine])
print(available_solar_for_BESS[checkLine])
print(BESS_charge_from_solar_pv[checkLine])

50.0
42.397456399999996
0.0


In [55]:
# BaseData = pd.DataFrame(columns=['year','month','day','day_of_year','hour_of_day','hour_of_year','load','Specific Solar PV Generation','Total Solar PV Generation','Solar PV Supply to Load','Available Solar PV for BESS','BESS Charge from Solar PV','Actual Solar PV Supply to BESS','BESS Capacity at Start of Day','Solar PV to be Curtailed'])
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.precision = 4
pd.set_option('display.precision',4)

BaseData  = pd.read_csv('DateTemplate.csv',header=0)
BaseData['load'] = customer_demand
BaseData['Specific Solar PV Generation'] = specific_solar_load.tolist()
BaseData['Total Solar PV Generation'] = total_solar_pv_generation.tolist()
BaseData['Solar PV Supply to Load'] = SPVS_to_load.tolist()
BaseData['Available Solar PV for BESS'] = available_solar_for_BESS.tolist()
BaseData['BESS Charge from Solar PV'] = BESS_charge_from_solar_pv.tolist()
BaseData['Actual Solar PV Supply to BESS'] = actual_solar_pv_supply_to_BESS.tolist()
BaseData['Solar PV to be Curtailed'] = solar_pv_to_be_curtailed.tolist()
BaseData['BESS Discharge to Load'] = BESS_discharge_to_load.tolist()
BaseData['Diesel Generator Supply'] = DG_supply.tolist()

# Get Totals
totals = pd.DataFrame({'Month':['Total'],'Day':['----'],'YearDay':['----'],'Hour':['---'],'load':[BaseData['load'].sum()],'Specific Solar PV Generation':[BaseData['Specific Solar PV Generation'].sum()],
'Total Solar PV Generation':[BaseData['Total Solar PV Generation'].sum()],'Solar PV Supply to Load':[BaseData['Solar PV Supply to Load'].sum()],'Available Solar PV for BESS':[BaseData['Available Solar PV for BESS'].sum()],'BESS Charge from Solar PV':[BaseData['BESS Charge from Solar PV'].sum()],'Actual Solar PV Supply to BESS':[BaseData['Actual Solar PV Supply to BESS'].sum()],'Solar PV to be Curtailed':[BaseData['Solar PV to be Curtailed'].sum()],'BESS Discharge to Load':[BaseData['BESS Discharge to Load'].sum()],'Diesel Generator Supply':[BaseData['Diesel Generator Supply'].sum()]},index=["Total"])

BaseData=BaseData.append(totals)
BaseData.to_csv('Results.csv',index=False)
print('Data Saved')

Data Saved


In [56]:
# Code for generating daily distribution

import plotly
import plotly.graph_objects as go

#Select Day of Year
oneDay = BaseData[(BaseData['YearDay'] == 2)]
fig = go.Figure()
fig.add_trace(go.Bar(x=oneDay['Hour'], y = oneDay['Solar PV Supply to Load'], name = "Solar PV"))
fig.add_trace(go.Bar(x=oneDay['Hour'], y = oneDay['BESS Discharge to Load'], name = "BESS"))
fig.add_trace(go.Bar(x=oneDay['Hour'], y = oneDay['Diesel Generator Supply'], name = "Diesel Generator"))
fig.update_layout(barmode='stack')
fig.show()